# Implementing transformers from scratch

## Imports

In [1]:
import torch
import torch.nn as nn

## Vanilla Self attention

In [2]:

class SelfAttention(nn.Module):
    def __init__(self, d_in: int, d_out: int, qkv_bias: bool = False):
        super().__init__()
        self.W_q = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_k = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_v = nn.Linear(d_in, d_out, bias = qkv_bias)
        
    def forward(self, x):
        keys = self.W_k(x)
        queries = self.W_q(x)
        values = self.W_v(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
            )
        return attn_weights @ values
        

## Casual attention (Masking future tokens)

In [3]:
class CasualAttention(nn.Module):
    def __init__(self, d_in: int, d_out: int, context_length: int, dropout: float, qkv_bias: bool = False):
        super().__init__()
        self.d_out = d_out
        self.W_q = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_k = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_v = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length),
            diagonal=1)
            ) 
    
    def forward(self, x):
        batch, num_tokens, d_in = x.shape
        keys = self.W_k(x)
        queries = self.W_q(x)
        values = self.W_v(x)
        attn_scores = queries @ keys.transpose(1,2)
        attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
            )
        attn_weights = self.dropout(attn_weights)
        return attn_weights @ values
        

## Multihead attention (multiple QKV pairs)

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in: int, d_out: int, context_length: int, dropout: float, num_heads: int, qkv_bias: bool = False):
        super().__init__()
        assert (d_out % num_heads == 0), \
        "d_out must be divisible by num_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_q = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_k = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_v = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length),
            diagonal=1)
            )
        
    def forward(self, x):
        batch, num_tokens, d_in = x.shape
        
        # Project input x with qkv weight matrices, each has dimensions: (batch, num_tokens, d_out)
        keys = self.W_k(x)
        queries = self.W_q(x)
        values = self.W_v(x) 
        
        # Unrolling the last dimension d_out --> (self.num_heads, self.head_dim), shape is now (b, num_tokens, num_heads, head_dim)
        keys = keys.view(batch, num_tokens, self.num_heads, self.head_dim)
        values = values.view(batch, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(
        batch, num_tokens, self.num_heads, self.head_dim
        )
        
        # Transposes the shape, shape is now (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)
        
        # Since the first 2 are basically "metadata", we transpose the dimensions of dim 2 and 3 instead (we padded an additional dimension "num heads" before the thing)
        attn_scores = queries @ keys.transpose(2, 3)
        
        # Masking future tokens (to prevent information leakage)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf) 
        
        attn_weights = torch.torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
            )
        
        # From this step onwards, reverts the "transformations" before the multihead attention operations occurs
        
        # Shape reverted  from (b, num_heads, num_tokens, head_dim) back to (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)
        
        # Combines the num_heads and head_dim dimensions back to the d_out dimension, 
        # We get the vector "back to its original format" (since we just performed)
        context_vec = context_vec.contiguous().view(
            batch, num_tokens, self.d_out
            )
        
        context_vec = self.out_proj(context_vec)
        return context_vec


## Multiquery Attention (shared KV matrices)

In [5]:
class MultiQueryAttention(nn.Module):
    def __init__(self, d_in: int, d_out: int, context_length: int, dropout: float, num_heads: int, qkv_bias: bool = False):
        super().__init__()
        assert (d_out % num_heads == 0), \
        "d_out must be divisible by num_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        
        # Query projects to full d_out dimension (will be split into heads)
        self.W_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        # Key and Value project only to head_dim (shared across heads)
        self.W_k = nn.Linear(d_in, self.head_dim, bias=qkv_bias)
        self.W_v = nn.Linear(d_in, self.head_dim, bias=qkv_bias)
        
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length),
            diagonal=1)
        )
        
    def forward(self, x):
        batch, num_tokens, d_in = x.shape
        
        # Project queries to full dimension and reshape
        queries = self.W_q(x).view(batch, num_tokens, self.num_heads, self.head_dim)
        # Project keys and values to single head dimension
        keys = self.W_k(x)    # shape: (batch, num_tokens, head_dim)
        values = self.W_v(x)  # shape: (batch, num_tokens, head_dim)
        
        # Transpose queries for attention computation
        queries = queries.transpose(1, 2)  # (batch, num_heads, num_tokens, head_dim)
        
        # Compute attention scores
        # keys/values are broadcast across num_heads dimension
        attn_scores = queries @ keys.unsqueeze(1).transpose(-2, -1)
        
        # Masking future tokens
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(
            attn_scores / self.head_dim**0.5, dim=-1
        )
        
        # Apply attention weights to values (broadcasting happens automatically)
        context_vec = (attn_weights @ values.unsqueeze(1)).transpose(1, 2)
        
        # Reshape back to original dimensions
        context_vec = context_vec.contiguous().view(batch, num_tokens, self.d_out)
        
        return self.out_proj(context_vec)

## Grouped Query Attention (multiple Q matrices, each shared by multiple kv pairs)

In [6]:
class GroupedQueryAttention(nn.Module):
    def __init__(self, d_in: int, d_out: int, context_length: int, dropout: float, 
                 num_heads: int, num_kv_heads: int, qkv_bias: bool = False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
        assert (num_heads % num_kv_heads == 0), "num_heads must be divisible by num_kv_heads"
        
        self.d_out = d_out
        self.num_heads = num_heads
        self.num_kv_heads = num_kv_heads
        self.head_dim = d_out // num_heads
        self.num_queries_per_kv = num_heads // num_kv_heads
        
        # Query projects to full d_out dimension
        self.W_q = nn.Linear(d_in, d_out, bias=qkv_bias)
        # Key and Value project to (num_kv_heads * head_dim)
        self.W_k = nn.Linear(d_in, self.num_kv_heads * self.head_dim, bias=qkv_bias)
        self.W_v = nn.Linear(d_in, self.num_kv_heads * self.head_dim, bias=qkv_bias)
        
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            'mask',
            torch.triu(torch.ones(context_length, context_length),
            diagonal=1)
        )
        
    def forward(self, x: torch.Tensor):
        batch, num_tokens, d_in = x.shape
        
        # Project and reshape queries
        queries = self.W_q(x).view(batch, num_tokens, self.num_heads, self.head_dim)
        
        # Project and reshape keys and values
        keys = self.W_k(x).view(batch, num_tokens, self.num_kv_heads, self.head_dim)
        values = self.W_v(x).view(batch, num_tokens, self.num_kv_heads, self.head_dim)
        
        # Transpose for attention computation
        queries = queries.transpose(1, 2)  # (batch, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)        # (batch, num_kv_heads, num_tokens, head_dim)
        values = values.transpose(1, 2)    # (batch, num_kv_heads, num_tokens, head_dim)
        
        # Repeat keys and values for each query in the group
        keys = keys.repeat_interleave(self.num_queries_per_kv, dim=1)
        values = values.repeat_interleave(self.num_queries_per_kv, dim=1)
        
        # Compute attention scores
        attn_scores = queries @ keys.transpose(-2, -1)
        
        # Masking future tokens
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(
            attn_scores / self.head_dim**0.5, dim=-1
        )
        
        # Apply attention weights
        context_vec = (attn_weights @ values).transpose(1, 2)
        
        # Reshape back to original dimensions
        context_vec = context_vec.contiguous().view(batch, num_tokens, self.d_out)
        
        return self.out_proj(context_vec)

# Building the transformer block

## Layernorm

In [7]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))
    
    def forward(self, x: torch.Tensor):
        mean = x.mean(dim = -1, keepdim = True)
        var = x.var(dim = -1, keepdim = True, unbiased = False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

## Gelu activation function

In [8]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
            ))

## FFN (with GELU activations)

In [9]:
class FeedForward(nn.Module):
    def __init__(self, emb_dim: int, projection_multiplier: float = 4.0):
        super().__init__()
        self.layers = nn.Sequential(
        nn.Linear(emb_dim, int(projection_multiplier*emb_dim)),
        GELU(),
        nn.Linear(int(projection_multiplier*emb_dim), emb_dim),
        )
    def forward(self, x):
        return self.layers(x)

## Transformer Block

In [10]:
class TransformerBlock(nn.Module):
    def __init__(self, emb_dim: int, context_length: int, n_heads: int, drop_rate: float, qkv_bias: bool, ff_projection_multiplier: float = 4.0, **kwargs):
        super().__init__()
        
        self.attention = MultiHeadAttention(
        d_in=emb_dim,
        d_out=emb_dim,
        context_length=context_length,
        num_heads=n_heads,
        dropout=drop_rate,
        qkv_bias=qkv_bias)
        
        self.feedforward = FeedForward(emb_dim, ff_projection_multiplier)
        self.norm1 = LayerNorm(emb_dim)
        self.norm2 = LayerNorm(emb_dim)
        self.drop_shortcut = nn.Dropout(drop_rate)
        
    def forward(self, x):
        
        shortcut = x
        x = self.norm1(x)
        x = self.attention(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        
        shortcut = x
        x = self.norm2(x)
        x = self.feedforward(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        
        return x

### Test Transformer Block

In [11]:
GPT_CONFIG_124M = {
    "vocab_size": 50257, # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768, # Embedding dimension
    "n_heads": 12, # Number of attention heads
    "n_layers": 12, # Number of layers
    "drop_rate": 0.1, # Dropout rate
    "qkv_bias": False # Query-Key-Value bias
}

In [12]:
torch.manual_seed(123)
x = torch.rand(2, 4, 768)
block = TransformerBlock(**GPT_CONFIG_124M)
output = block(x)

print("Input shape:", x.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])


## GPT model

In [15]:
class GPTModel(nn.Module):
    def __init__(self, vocab_size: int, emb_dim: int, context_length: int, n_heads: int, n_layers: int, drop_rate: float, qkv_bias: bool, ff_projection_multiplier: float = 4.0, **kwargs):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, emb_dim)
        self.positional_emb = nn.Embedding(context_length, emb_dim)
        self.drop_emb = nn.Dropout(drop_rate)

        self.transformer_blocks = nn.Sequential(
        *[TransformerBlock(emb_dim, context_length, n_heads, drop_rate, qkv_bias, ff_projection_multiplier) for _ in range(n_layers)]
        )

        self.final_norm = LayerNorm(emb_dim)
        self.out_head = nn.Linear(
        emb_dim, vocab_size, bias=False
        )
    def forward(self, in_idx):
        
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.token_emb(in_idx)

        pos_embeds = self.positional_emb(
        torch.arange(seq_len, device=in_idx.device)
        )
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

### Test GPT model block

In [ ]:
GPT_CONFIG_124M = {
    "vocab_size": 50257, # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768, # Embedding dimension
    "n_heads": 12, # Number of attention heads
    "n_layers": 12, # Number of layers
    "drop_rate": 0.1, # Dropout rate
    "qkv_bias": False # Query-Key-Value bias
}

In [18]:
torch.manual_seed(123)
input_batch = torch.tensor([[6109, 3626, 6100, 345],
                             [6109, 1110, 6622, 257]], dtype=torch.long)
model = GPTModel(**GPT_CONFIG_124M)
output = model(input_batch)

print("Input shape:", input_batch.shape)
print("Output shape:", output.shape)
print(output)

Input shape: torch.Size([2, 4])
Output shape: torch.Size([2, 4, 50257])
tensor([[[ 0.1936,  0.1579, -0.1142,  ...,  0.2896, -0.0771, -0.2860],
         [ 0.4969, -0.4337, -0.5846,  ..., -0.3765, -0.0187, -0.2397],
         [ 0.9601, -0.1557,  0.1358,  ...,  0.0581, -0.4173, -0.1250],
         [-0.7732,  0.0467, -0.3654,  ...,  0.8903,  0.3966, -0.4360]],

        [[ 0.1809,  0.3011, -0.2012,  ...,  0.1549,  0.2090, -0.4654],
         [ 0.2816, -0.1873,  0.1192,  ...,  1.2138,  0.0332,  0.1018],
         [ 0.7811,  0.7691, -0.2902,  ...,  0.7433,  0.3737, -0.0160],
         [-0.1014, -0.3333,  0.2874,  ...,  1.1855, -0.6303,  0.0312]]],
       grad_fn=<UnsafeViewBackward0>)


In [19]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 163,009,536


In [20]:
total_size_bytes = total_params * 4
total_size_mb = total_size_bytes / (1024 * 1024)
print(f"Total size of the model: {total_size_mb:.2f} MB")

Total size of the model: 621.83 MB


## Text Generation

### Greedy Decoding

In [ ]:
def generate_text_greedy(model: nn.Module, idx: torch.Tensor, max_new_tokens: int, context_size: int):
    
    # Generates "max_new_tokens" tokens from the model, no stopping mechanism using special "<|endoftext|>" tokens
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)

        logits = logits[:, -1, :] # check only the logits of the last word
        probas = torch.softmax(logits, dim=-1)
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)
        idx = torch.cat((idx, idx_next), dim=1)
    
    return idx

In [ ]:
from typing import Optional

def generate_text(model: nn.Module, idx: torch.Tensor, max_new_tokens: int, context_size: int, temperature: float = 0.0, top_p: float = 1.0, top_k: Optional[int] = None, eos_id: Optional[int] = None):
    
    # Generates "max_new_tokens" tokens from the model, no stopping mechanism using special "<|endoftext|>" tokens
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)

        logits = logits[:, -1, :] # check only the logits of the last word
        
        if temperature == 0:  # Better to check for 0 explicitly
            logits = logits  # No temperature scaling
        else:
            logits = logits / temperature
        
        if top_k is not None:
            _, top_k_pos = torch.topk(logits, top_k) # Find top_k positions
            
            # Create a boolean mask for positions
            indices = torch.arange(logits.size(-1), device=logits.device)
            is_in_topk = torch.isin(indices, top_k_pos, assume_unique = True) # assume_unique = True as indices won't repeat.
            
            # Only keep top k and change the rest into inf
            logits = torch.where(
                condition=is_in_topk,
                input=logits,
                other=torch.tensor(float('-inf'), device=logits.device)
            )
        
        probas = torch.softmax(logits, dim=-1)
        
        if top_p <= 1.0:
            sorted_probas, _ = torch.sort(probas, descending=True)
            
            # Cumulative sum according by logits, and then mask logits if accumulated over topp
            cumulative_probas = torch.cumsum(sorted_probas, dim=-1) 
            mask = cumulative_probas <= top_p
            
            cutoff_idx = torch.nonzero(mask)[-1]
            cutoff = sorted_probas[cutoff_idx]
            
            probas = torch.where(probas < cutoff, torch.tensor(0.0), probas)
            probas = probas / probas.sum()
        
        # # What the fuck does torch scatter/ torch gather do.
        # if top_p < 1.0:  # Only apply if top_p < 1.0
        #     sorted_probas, sorted_indices = torch.sort(probas, descending=True)
        #     cumulative_probas = torch.cumsum(sorted_probas, dim=-1)
        #     # Remove tokens with cumulative probability above the threshold
        #     sorted_indices_to_remove = cumulative_probas > top_p
        #     # Shift the indices to the right to keep also the first token above the threshold
        #     sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        #     sorted_indices_to_remove[..., 0] = 0

        #     indices_to_remove = sorted_indices_to_remove.scatter(dim=-1, index=sorted_indices, src=sorted_indices_to_remove)
        #     probas = probas.masked_fill(indices_to_remove, 0.0)
        #     probas = probas / probas.sum(dim=-1, keepdim=True)  # renormalize
            
        if temperature > 0:
            idx_next = torch.multinomial(probas, dim=-1, keepdim=True)  # rolls according to the value of each face
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)
        
        idx = torch.cat((idx, idx_next), dim=1)
        
        if eos_id is not None and (idx_next == eos_id).any():  # Check any occurrence of EOS
            break
        
    return idx